In [1]:
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_statement_excel_file(input_file_path, header_name='Place_Value'):
    # Read the data into a pandas DataFrame
    data = pd.read_excel(input_file_path)
    # Drop the specified row, set the header, drop unnecessary rows, and remove columns with all NaN values
    data = data.drop(1)
    data.columns = data.iloc[0]
    data = data.drop(0)
    data = data.drop(data.index[-1])
    columns_to_replace_null = ['Instr ', 'Instr No']
    data[columns_to_replace_null] = data[columns_to_replace_null].fillna(0)
    data = data.dropna(axis=1, how='all')
    # Assign the specified header name to columns with NaN header
    data.columns = [header_name if pd.isna(col) else col for col in data.columns]
    return data

In [3]:
def expand_data(data, header_name='Place_Value'):
    # Splitting Tran Particulars
    split_columns = data['Tran Particulars'].str.split('/', expand=True)
    split_columns.columns = [str(i) for i in range(1, len(split_columns.columns) + 1)]
    data = pd.concat([data, split_columns], axis=1)
    # Re-ordering columns
    data = data[['Tran Date', 'Value Date', 'Tran Particulars', '1', '2', '3', '4', '5', 'Type', 'Sub Type', 'Instr ', 'Instr No', 'Debit', 'Credit', 'Balance', 'Place_Value']]
    # Rename columns
    data.rename(columns={'1': 'Particulars_1', '2': 'Particulars_2', '3': 'Particulars_3', '4': 'Particulars_4', '5': 'Particulars_5'}, inplace=True)
    # Sort by 'Particulars_1'
    data = data.sort_values(by='Particulars_1')
    # Condition for 'SBINT:'
    condition = data['Particulars_1'].str.contains(r'SBINT:')
    data['Particulars_1_Updated'] = np.where(condition, 'SBIINT', data['Particulars_1'])
    data['Duration_Intrest_Credited'] = np.where(
        condition,
        data['Particulars_1'].str.extract(r'SBINT:(.*)', expand=False),
        np.nan
    )
    # Read MCC codes data
    data_codes = pd.read_excel('mcc_data.xlsx')
    data_codes['CODE'] = data_codes['CODE'].astype(str)
    def match_code_merchant_group(row):
        # Check each particulars column for a match in data_codes
        for col in ['Particulars_1', 'Particulars_2', 'Particulars_3', 'Particulars_4', 'Particulars_5']:
            code = row[col]
            if pd.isnull(code):
                continue
            # If the value in the cell is a string with length 4, then it might be a code
            if isinstance(code, str) and len(code) == 4:
                match = data_codes[data_codes['CODE'] == code]
                if not match.empty:
                    return match.iloc[0]['CODE_DESC'], match.iloc[0]['MERCHANT'], match.iloc[0]['GROUP_CATEGORY_NEW']
        return None, None, None
    # Create new columns for 'MERCHANT' and 'GROUP_CATEGORY_NEW'
    data['CODE_DESC'] = None
    data['MERCHANT'] = None
    data['GROUP_CATEGORY_NEW'] = None
    # Apply the function to each row in the dataframe
    for index, row in data.iterrows():
        code_desc, merchant, group_category_new = match_code_merchant_group(row)
        data.at[index, 'CODE_DESC'] = code_desc
        data.at[index, 'MERCHANT'] = merchant
        data.at[index, 'GROUP_CATEGORY_NEW'] = group_category_new
    grouped_data = data.groupby('Particulars_1_Updated')
    dataframes = {name: group for name, group in grouped_data}
    data['Tran Date'] = pd.to_datetime(data['Tran Date'])
    data['Value Date'] = pd.to_datetime(data['Value Date'])
    data['Tran Date'].dt.strftime('%Y-%m-%d')
    data['Value Date'].dt.strftime('%Y-%m-%d')
    return data

In [4]:
def acc_statement_Report(data, output_txt_filename='acc_statement_report.txt'):
    total_transactions = len(data)
    print(f'Total Transaction Count: {total_transactions}')
    # Convert 'Tran Date' column to datetime
    data['Tran Date'] = pd.to_datetime(data['Tran Date'])
    start_date = data['Tran Date'].min()
    end_date = data['Tran Date'].max()
    print(f'Statement Period: {start_date.strftime("%Y-%m-%d")} to {end_date.strftime("%Y-%m-%d")}')
    # Calculate the number of months within the statement period
    months_difference = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month
    print(f'Number of Months within the Period: {months_difference}')
    tc_values_counts = data['Particulars_1_Updated'].value_counts()
    print("----------------------------------")
    print("Count of each Transaction Category")
    print("----------------------------------")
    print(tc_values_counts)
    print("----------------------------------")
    type_values_counts = data['Type'].value_counts()
    print("Count of each Type")
    print("----------------------------------")
    print(type_values_counts)
    print("----------------------------------")
    sub_type_values_counts = data['Sub Type'].value_counts()
    print("Count of each Sub Type")
    print("----------------------------------")
    print(sub_type_values_counts)
    credit_count = data['Credit'].notnull().sum()
    debit_count = data['Debit'].notnull().sum()
    print("----------------------------------")
    print(f'Total Credit Transactions: {credit_count}')
    print(f'Total Debit Transactions: {debit_count}')
    print("----------------------------------")
    grouped_data = data.groupby('Particulars_1_Updated')[['Credit', 'Debit']].count()
    print("----------------------------------")
    print("Count of Credit and Debit Transactions for each Transaction Category:")
    print(grouped_data)
    print("----------------------------------")
##############################################################################################################
    with open(output_txt_filename, 'w') as file:
        file.write(f'Total Transaction Count: {total_transactions}\n')
        file.write(f'Statement Period: {start_date.strftime("%Y-%m-%d")} to {end_date.strftime("%Y-%m-%d")}\n')
        file.write(f'Number of Months within the Period: {months_difference}\n')
        file.write("----------------------------------\n")
        file.write("\nCount of each Transaction Category\n")
        file.write("----------------------------------\n")
        file.write(f"{tc_values_counts}\n")

        file.write("----------------------------------\n")
        file.write("\nCount of each Type\n")
        file.write("----------------------------------\n")
        file.write(f"{type_values_counts}\n")

        file.write("----------------------------------\n")
        file.write("\nCount of each Sub Type\n")
        file.write("----------------------------------\n")
        file.write(f"{sub_type_values_counts}\n")

        file.write("----------------------------------\n")
        file.write(f'Total Credit Transactions: {credit_count}\n')
        file.write(f'Total Debit Transactions: {debit_count}\n')
        file.write("----------------------------------\n")

        file.write("----------------------------------\n")
        file.write(f'Count of Credit and Debit Transactions for each Transaction Category: {grouped_data}\n')
        file.write("----------------------------------\n")

    print(f'Results saved to {output_txt_filename}')

In [5]:
def calculate_credit_debit_counts(data):
  credit_count = data['Credit'].notnull().sum()
  debit_count = data['Debit'].notnull().sum()
  print(f'Total Credit Transactions: {credit_count}')
  print(f'Total Debit Transactions: {debit_count}')

In [6]:
def calculate_credit_debit_counts_by_transaction_category(data):
  grouped_data = data.groupby('Particulars_1_Updated')[['Credit', 'Debit']].count()
  print("Count of Credit and Debit Transactions for each Category:")
  print(grouped_data)

In [7]:
def separate_single_multi_transaction_per_days(data):
  # Convert 'Tran Date' column to datetime
  data['Tran Date'] = pd.to_datetime(data['Tran Date'])
  # Count the number of transactions for each 'Tran Date'
  transaction_counts = data['Tran Date'].value_counts()
  # Get 'Tran Date' with multiple transactions
  multiple_transactions_dates = transaction_counts[transaction_counts > 1].index
  # Separate data into two tables
  single_transaction_table = data[data['Tran Date'].isin(multiple_transactions_dates) == False]
  multiple_transactions_table = data[data['Tran Date'].isin(multiple_transactions_dates)]
  # Sort tables based on 'Tran Date'
  single_transaction_table = single_transaction_table.sort_values(by='Tran Date')
  multiple_transactions_table = multiple_transactions_table.sort_values(by='Tran Date')
  return single_transaction_table, multiple_transactions_table

In [8]:
def separate_credit_debit(data):
  # Separate data into two tables
  credit_table = data[data['Credit'].notnull()]
  debit_table = data[data['Debit'].notnull()]
  total_credit = credit_table['Credit'].sum()
  total_debit = debit_table['Debit'].sum()
  print(f'Total Credit: {total_credit}')
  print(f'Total Debit: {total_debit}')
  return credit_table, debit_table

In [9]:
def divide_data_by_tran_categories(data, particular_value):
  unique_values = data['Particulars_1_Updated'].unique()
  if particular_value in unique_values:
      filtered_data = data[data['Particulars_1_Updated'] == particular_value]
      return filtered_data

In [10]:
def divide_data_by_months(data, target_month):
  # Convert 'Tran Date' column to datetime
  data['Tran Date'] = pd.to_datetime(data['Tran Date'])
  # Extract unique months from 'Tran Date'
  unique_months = data['Tran Date'].dt.to_period('M').unique()
  if target_month in unique_months:
      # Filter data based on the specified month
      filtered_data = data[data['Tran Date'].dt.to_period('M') == target_month]
      return filtered_data
# target_month=yyyy-mm format

In [11]:
def opening_closing_balance_month(data):
    # Convert 'Tran Date' column to datetime
    data['Tran Date'] = pd.to_datetime(data['Tran Date'])
    # Extract unique months from 'Tran Date'
    unique_months = data['Tran Date'].dt.to_period('M').unique()
    # Initialize an empty DataFrame for the summary table
    summary_table = pd.DataFrame(columns=['Month', 'Opening Balance', 'Closing Balance', 'Balance Difference'])
    for month in unique_months:
        # Filter data for the current month
        month_data = data[data['Tran Date'].dt.to_period('M') == month]
        if not month_data.empty:
            # Calculate opening balance (balance on the first date of the month)
            opening_balance = month_data.sort_values(by='Tran Date')['Balance'].iloc[0]
            # Calculate closing balance (balance on the last date of the month)
            closing_balance = month_data.sort_values(by='Tran Date')['Balance'].iloc[-1]
            # Calculate the difference between opening and closing balances
            balance_difference = closing_balance - opening_balance
           # Append the summary for the current month to the summary table
            summary_table = summary_table.append({
                'Month': str(month),
                'Opening Balance': opening_balance,
                'Closing Balance': closing_balance,
                'Balance Difference': balance_difference
            }, ignore_index=True)
    # Sort the summary table by 'Month'
    summary_table['Month'] = pd.to_datetime(summary_table['Month'])
    summary_table = summary_table.sort_values(by='Month')
    return summary_table

In [12]:
def add_credit_amount_bin_column(data):
    # Define the credit amount bins
    bins = [float('-inf'), 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000,
            5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000,
            20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, float('inf')]
    # Define the labels for each bin
    labels = ['<500', '500-1000', '1000-1500', '1500-2000', '2000-2500', '2500-3000', '3000-3500', '3500-4000',
              '4000-4500', '4500-5000', '5000-5500', '5500-6000', '6000-6500', '6500-7000', '7000-7500', '7500-8000',
              '8000-8500', '8500-9000', '9000-9500', '9500-10000', '10000-20000', '20000-30000', '30000-40000',
              '40000-50000', '50000-60000', '60000-70000', '70000-80000', '80000-90000', '90000-100000', '>100000']
    # Add a new column 'Credit Amount Bin' based on the specified ranges
    data['Credit Amount Bin'] = pd.cut(data['Credit'], bins=bins, labels=labels, right=False)
    return data

In [13]:
def add_debit_amount_bin_column(data):
    # Define the credit amount bins
    bins = [float('-inf'), 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000,
            5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000,
            20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, float('inf')]
    # Define the labels for each bin
    labels = ['<500', '500-1000', '1000-1500', '1500-2000', '2000-2500', '2500-3000', '3000-3500', '3500-4000',
              '4000-4500', '4500-5000', '5000-5500', '5500-6000', '6000-6500', '6500-7000', '7000-7500', '7500-8000',
              '8000-8500', '8500-9000', '9000-9500', '9500-10000', '10000-20000', '20000-30000', '30000-40000',
              '40000-50000', '50000-60000', '60000-70000', '70000-80000', '80000-90000', '90000-100000', '>100000']
    # Add a new column 'Credit Amount Bin' based on the specified ranges
    data['Debit Amount Bin'] = pd.cut(data['Debit'], bins=bins, labels=labels, right=False)
    return data

In [14]:
def get_credit_amount_bin_counts(data):
  # Get counts of each bin in the 'Credit Amount Bin' column
  bin_counts = data['Credit Amount Bin'].value_counts().sort_index()
  # Create a DataFrame to display the counts
  counts_df = pd.DataFrame({'Credit Amount Bin': bin_counts.index, 'Count': bin_counts.values})
  return counts_df

In [15]:
def get_debit_amount_bin_counts(data):
  # Get counts of each bin in the 'Debit Amount Bin' column
  bin_counts = data['Debit Amount Bin'].value_counts().sort_index()
  # Create a DataFrame to display the counts
  counts_df = pd.DataFrame({'Debit Amount Bin': bin_counts.index, 'Count': bin_counts.values})
  return counts_df

In [16]:
def calculate_average_monthly_balance(data):
  # Convert 'Tran Date' column to datetime
  data['Tran Date'] = pd.to_datetime(data['Tran Date'])
  # Extract unique months from 'Tran Date'
  unique_months = data['Tran Date'].dt.to_period('M').unique()
  # Initialize an empty DataFrame for the average monthly balance
  amb_table = pd.DataFrame(columns=['Month', 'AMB'])
  for month in unique_months:
      # Filter data for the current month
      month_data = data[data['Tran Date'].dt.to_period('M') == month]
      # Calculate average monthly balance
      amb_value = month_data['Balance'].mean()
      # Append the result to the average monthly balance table
      amb_table = amb_table.append({
          'Month': str(month),
          'AMB': amb_value
      }, ignore_index=True)
  # Sort the table by 'Month'
  amb_table['Month'] = pd.to_datetime(amb_table['Month'])
  amb_table = amb_table.sort_values(by='Month')
  return amb_table

In [17]:
def calculate_average_monthly_balance_with_counts_and_total_amount(data):
  # Convert 'Tran Date' column to datetime
  data['Tran Date'] = pd.to_datetime(data['Tran Date'])
  # Extract unique months from 'Tran Date'
  unique_months = data['Tran Date'].dt.to_period('M').unique()
  # Initialize an empty DataFrame for the extended monthly balance table
  extended_amb_table = pd.DataFrame(columns=['Month', 'AMB', 'Credit Count', 'Debit Count', 'Total Amount Credited', 'Total Amount Debited', 'Total Amount'])
  for month in unique_months:
      # Filter data for the current month
      month_data = data[data['Tran Date'].dt.to_period('M') == month]
      # Calculate average monthly balance
      amb_value = month_data['Balance'].mean()
      # Calculate credit count and total amount credited
      credit_count = month_data['Credit'].count()
      total_amount_credited = month_data['Credit'].sum()
      # Calculate debit count and total amount debited
      debit_count = month_data['Debit'].count()
      total_amount_debited = month_data['Debit'].sum()
      # Calculate total amount (sum of total credit and total debit)
      total_amount = total_amount_credited + total_amount_debited
      # Append the result to the extended monthly balance table
      extended_amb_table = extended_amb_table.append({
          'Month': str(month),
          'AMB': amb_value,
          'Credit Count': credit_count,
          'Debit Count': debit_count,
          'Total Amount Credited': total_amount_credited,
          'Total Amount Debited': total_amount_debited,
          'Total Amount': total_amount
      }, ignore_index=True)
  # Sort the table by 'Total Amount'
  extended_amb_table['Total Amount'] = pd.to_numeric(extended_amb_table['Total Amount'])
  extended_amb_table = extended_amb_table.sort_values(by='Total Amount')
  return extended_amb_table

In [20]:
file='1.xlsx'
df=preprocess_statement_excel_file(file)

In [21]:
df.to_csv('csv.csv', index=False)

In [ ]:
df=expand_data(df)

In [ ]:
def filter_high_credit_transactions_quantile(data, column_name):
  # Calculate the credit value threshold based on the specified quantile
  q1 = data[column_name].quantile(.25)
  q3 = data[column_name].quantile(.75)
  iqr=q3-q1
  credit_threshold = q3+1.5*iqr
  # Filter transactions where credit value is higher than the threshold
  high_credit_transactions = data[data['Credit'] > credit_threshold]
  high_credit_transactions = high_credit_transactions.sort_values(by='Credit')
  columns_to_keep = ['Tran Date', 'Tran Particulars', 'Credit']
  high_credit_transactions = high_credit_transactions[columns_to_keep]
  return high_credit_transactions


column_name = 'Credit'  # Replace with the column name you want to filter
high_credit_transactions = filter_high_credit_transactions_quantile(df, column_name)
# high_credit_transactions.to_csv('high_credit_transactions_quantile.csv', index=False)


In [ ]:
high_credit_transactions

,Tran Date,Tran Particulars,Credit
271,2023-06-07,IFN/FBLEPIFIuaAa5u11i49kEei78ELe xxxxxxxx0111,1855.34
443,2023-10-16,MB FTO/232899176948/RAFEEK R/null/,1910
507,2024-01-25,UPI IN/402529794836/rafeekrahim25102000@oksb/0000,2000
61,2023-02-28,UPI IN/305981814413/dhananjudhananjay@oksbi//0000,2000
481,2023-12-21,UPI IN/335535831567/rafeekrahim25102000-2@ok/0000,2000
489,2024-01-01,IFN/FBLEPIFIyof6aEErmGC5vVb1ZrE4 xxxxxxxx0111,2050
310,2023-08-23,IFN/FBLEPIFICBzVgPnRxp67ATudNqyn xxxxxxxx0111,2935
74,2023-03-01,UPI IN/306029201187/akihdarsg7@okaxis/UPI/0000,4000
221,2023-05-02,UPI IN/348801220615/shifanasarv2001@okicici//0000,5333.34
232,2023-05-11,IFN/FBLEPIFIRRNdxRDwy5qqSMHfXEe9 xxxxxxxx0111,10000


In [ ]:
import plotly.express as px
fig = px.box(df, y="Credit")
fig.show()

In [ ]:
fig = px.box(df, x="Credit")
fig.show()

In [ ]:
def aaa(input_file_path):
    # Read the data into a pandas DataFrame
    data = pd.read_excel(input_file_path)
    return data

In [ ]:
file='aaa.xlsx'
aa=aaa(file)

In [ ]:
aa

,Tran Date,Value Date,Unnamed: 2,Tran Particulars,Type,Sub Type,Instr,Instr No,Debit,Credit,Balance,Unnamed: 11
0,2022-11-15,2022-11-15,NaN,IFN/FBLEPIFIbXA2xdScupcEednu45HV xxxxxxxx0111,T,CI,NaN,NaN,NaN,100.0,100.0,Cr
1,2022-11-15,2022-11-15,NaN,UPIOUT/231907746601/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,99.0,NaN,1.0,Cr
2,2022-11-15,2022-11-16,NaN,UPI IN/232018997547/navarasparayil@okhdfcban/0000,T,CI,NaN,NaN,NaN,200.0,201.0,Cr
3,2022-11-18,2022-11-18,NaN,UPIOUT/232215138698/rafipattazhi123@oksbi/UP/0000,T,CI,NaN,NaN,160.0,NaN,41.0,Cr
4,2022-11-18,2022-11-18,NaN,IFN/FBLEPIFIAUxAsBF8qm8VzgxzNVvd xxxxxxxx0111,T,CI,NaN,NaN,NaN,1010.0,1051.0,Cr
...,...,...,...,...,...,...,...,...,...,...,...,...
504,2024-01-23,2024-01-23,NaN,UPI IN/402324732873/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,NaN,100.0,100.0,Cr
505,2024-01-25,2024-01-25,NaN,UPI IN/402529794836/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,NaN,2000.0,2100.0,Cr
506,2024-01-25,2024-01-25,NaN,TO ECM/402515108810/CHATGPT SUBSCRIPTION \CHA...,T,CI,NaN,NaN,2021.3,NaN,78.7,Cr
507,2024-01-27,2024-01-27,NaN,UPIOUT/402791405164/sibinrajsibin3691@oksbi//0000,T,CI,NaN,NaN,60.0,NaN,18.7,Cr


In [ ]:
import pandas as pd

def find_repeating_transactions_across_months(data):
    # Convert 'Tran Date' column to datetime
    data['Tran Date'] = pd.to_datetime(data['Tran Date'])

    # Group by transaction characteristics excluding 'Month'
    grouped_data = data.groupby(['Tran Particulars', 'Type', 'Sub Type'] + [col for col in data.columns if col not in ['Month']])

    # Identify transactions with the same characteristics that occur in different months
    repeating_transactions = grouped_data.filter(lambda x: len(x['Month'].unique()) > 1)

    return repeating_transactions

repeating_transactions = find_repeating_transactions_across_months(aa)
# repeating_transactions.to_csv('repeating_transactions_across_months.csv', index=False)


In [ ]:
s,m=separate_single_multi_transaction_per_days(aa)

In [ ]:
s

,Tran Date,Value Date,Unnamed: 2,Tran Particulars,Type,Sub Type,Instr,Instr No,Debit,Credit,Balance,Unnamed: 11
25,2022-11-27,2022-11-27,NaN,UPIOUT/233108642912/akashagrawal686@okicici//0000,T,CI,NaN,NaN,40.00,NaN,248.00,Cr
26,2022-11-29,2022-11-29,NaN,UPIOUT/233305204324/paytmqr281005050101lty39/5411,T,CI,NaN,NaN,170.00,NaN,78.00,Cr
27,2022-12-01,2022-12-01,NaN,IFN/FBLEPIFIyG82kpfqujpRndqLf75W xxxxxxxx0111,T,CI,NaN,NaN,NaN,422.0,500.00,Cr
28,2022-12-02,2022-12-02,NaN,UPIOUT/233672511407/zomato-order@paytm/Zomat/5812,T,CI,NaN,NaN,248.75,NaN,251.25,Cr
35,2022-12-09,2022-12-09,NaN,UPIOUT/234310324152/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,27.00,NaN,602.25,Cr
38,2022-12-11,2022-12-11,NaN,IFN/FBLEPIFISJeNi8koG112JKEjtCLV xxxxxxxx0111,T,CI,NaN,NaN,NaN,19.0,410.25,Cr
39,2022-12-18,2022-12-18,NaN,UPIOUT/232216458473/9048682885@jupiteraxis/U/0000,T,CI,NaN,NaN,50.00,NaN,1001.00,Cr
40,2022-12-24,2022-12-23,NaN,SBINT:15-11-2022 to 23-12-2022[55550123400796],T,IP,NaN,NaN,NaN,1.0,11.25,Cr
49,2023-01-14,2023-01-14,NaN,UPIOUT/301406350466/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,545.00,NaN,0.25,Cr
50,2023-01-17,2023-01-17,NaN,IFN/FBLEPIFIRpRRbpepGvoqeHMBryHj xxxxxxxx0111,T,CI,NaN,NaN,NaN,120.0,120.25,Cr


In [ ]:
m

,Tran Date,Value Date,Unnamed: 2,Tran Particulars,Type,Sub Type,Instr,Instr No,Debit,Credit,Balance,Unnamed: 11
0,2022-11-15,2022-11-15,NaN,IFN/FBLEPIFIbXA2xdScupcEednu45HV xxxxxxxx0111,T,CI,NaN,NaN,NaN,100.0,100.0,Cr
1,2022-11-15,2022-11-15,NaN,UPIOUT/231907746601/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,99.0,NaN,1.0,Cr
2,2022-11-15,2022-11-16,NaN,UPI IN/232018997547/navarasparayil@okhdfcban/0000,T,CI,NaN,NaN,NaN,200.0,201.0,Cr
3,2022-11-18,2022-11-18,NaN,UPIOUT/232215138698/rafipattazhi123@oksbi/UP/0000,T,CI,NaN,NaN,160.0,NaN,41.0,Cr
4,2022-11-18,2022-11-18,NaN,IFN/FBLEPIFIAUxAsBF8qm8VzgxzNVvd xxxxxxxx0111,T,CI,NaN,NaN,NaN,1010.0,1051.0,Cr
...,...,...,...,...,...,...,...,...,...,...,...,...
501,2024-01-18,2024-01-18,NaN,UPIOUT/401862231639/bharatpe.90066447882@fbp/5814,T,CI,NaN,NaN,15.0,NaN,282.0,Cr
502,2024-01-18,2024-01-18,NaN,UPIOUT/401863283665/q323657886@ybl/UPI/5814,T,CI,NaN,NaN,20.0,NaN,262.0,Cr
503,2024-01-18,2024-01-18,NaN,UPIOUT/401869823827/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,262.0,NaN,0.0,Cr
505,2024-01-25,2024-01-25,NaN,UPI IN/402529794836/rafeekrahim25102000@oksb/0000,T,CI,NaN,NaN,NaN,2000.0,2100.0,Cr
